# Basic Condorcet

For a quick test, let's look at basic Condorcet voting. Recall that Condorcet looks for the option that wins all pairwise majority elections against every other option. Consider the set of agents $N = \{ A, B, C, D\}$ voting over solutions $\{1, 2, 3, 4\}$ and the following preference profile:

<div>
<img src="preference_profile.png" width="300"/>
</div>

Investigating all of the pairwise duels in a matrix, we'd get (for example, the three agents $A, B, C$ like 1 better than 2, only $D$ prefers 2 to 1, so 1 wins 3:1 over 2) : 

<div>
<img src="duels.png" width="500"/>
</div>

And option 4 is the Condorcet winner that we'd hope to find. I've used numbers for options to make the transition to MiniZinc a bit easier (maybe we turn to enums a,b,c for better illustration).


## MiniZinc basic toy example

I wanted to abstract away some of the complications from directly considering solutions but also did not only want to vote over the value of a single integer variable, so only some combinationsof variable assignments $x$ and $y$ are valid and these get arbitrarily numbered (indicated by the variable `control`):

<div>
<img src="sols.png" width="300"/>
</div>

That at least keeps the illusion of voting over solutions to the CSP.
```
constraint if control = 1 then x = 1 /\ y = 3 endif;
constraint if control = 2 then x = 2 /\ y = 2 endif;
constraint if control = 3 then x = 1 /\ y = 2 endif;
constraint if control = 4 then x = 3 /\ y = 1 endif;
```

Now the above encoding of agents' preferences can simply written as an array of integers denoting the ordering of control values.

<div>
<img src="preference_profile.png" width="200"/>
</div>

```
array[AGENTS,CHOICES] of CHOICES: prefs = 
[| 1, 2, 4, 3 | 4, 1, 3, 2 | 3, 4, 1, 2 | 4, 2, 3, 1 |];
```

The full [MiniZinc model](base_model_with_prefs.mzn)
 does not much more other than providing a rank for every agent how they perceive the current solution according to prefs. So, if `control = 2` then `rank[1] = 2`, `rank[2] = 4`, etc.

```
array[AGENTS] of var 1..n_options: rank;

constraint forall(a in AGENTS) (
  prefs[a,rank[a]] = control
);
```

which we will use to post the Condorcet-improvement criterion during custom Branch-and-Bound.

## The Meta-Search

Basically, everything works just as in normal branch-and-bound for now, but instead of posting 

```
child.add_string(f"constraint obj > {res['obj']}")
```

I would post the following (now in prose):

```
Given the current solution that we're at, give me the next solution that has better ranks (is liked better) by more than (or equal to) half the agents:
```

Let's implement this directly

In [3]:
import minizinc
from minizinc import Instance, Model, Result, Solver, Status
import asyncio

gecode = Solver.lookup("gecode")
m = Model("base_model_with_prefs.mzn")
inst = Instance(gecode, m)

res: Result = inst.solve()
print(res.solution)
while res.status == Status.SATISFIED:
    with inst.branch() as child:
        child.add_string("array[AGENTS] of 1..n_options+1: old_rank;")
        child["old_rank"] = res["rank"]  # copy the current ranks
        child.add_string("constraint sum(a in AGENTS) ( bool2int(rank[a] < old_rank[a] ) ) > win_thresh;")

        res = child.solve()
        if res.solution is not None:
            print(res.solution)

RuntimeError: asyncio.run() cannot be called from a running event loop